# Stilometrie: basale Teststrategien und grundlegende Ansätze

Das Notebook zeigt grundlegende und einfache Teststrategien, um über stilometrische Ansätze, Autor:innen zu identifizieren.

Folgende Verfahren werden getestet:

- Häufigkeitsverteilung der Wortlänge
- Häufigkeitsverteilung der Stoppwörter
- Häufigkeitsverteilung der Wortarten (Part-of-Speech)
- Vergleich der am häufigsten verwendeten Wörter
- Jaccard Similarity

Der nachfolgende Python-Code basiert auf zwei Vorlagen, der für das vorliegende Notebook angepasst wurde:

Lee Vaughan, Real World Python: A Hacker’s Guide to Solving Problems with Code, San Francisco 2021, S. 27-50. Der zum Buch gehörige Code unter: https://github.com/rlvaugh/Real_World_Python/blob/master/Chapter_2/stylometry.py

François Dominic Laramée, Introduction to stylometry with Python, in: Programming Historian 7 (2018), https://doi.org/10.46430/phen0078.

## Importe

In [ ]:
import nltk

from nltk.corpus import stopwords

import pandas as pd
import matplotlib.pyplot as plt
import sklearn.feature_extraction.text as text

%matplotlib inline

In [ ]:
# Folgende Daten müssen einmalig nach der Installation von NLTK heruntergeladen werden:

# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')

## Helferfunktionen 

### Einlesen der Dateien

In [ ]:
def text_to_string(filename):
    '''Read a text file and return a string.'''

    with open(f'data/{filename}') as f:
        return f.read()

### Erstellen eines Dictionaries mit tokenisierten Wörtern

In [ ]:
def make_word_dict(strings_by_author):
    """Return dictionary of tokenized words by corpus by author."""
    words_by_author = dict()
    for author in strings_by_author:
        tokens = nltk.word_tokenize(strings_by_author[author], language='german')
        words_by_author[author] = ([token.lower() for token in tokens if token.isalpha()])
    return words_by_author

### Kürzesten Corpus finden

Bei den stilometrischen Tests sollten die zu vergleichenden Texte die gleiche Länge aufweisen. Es ist möglich mit Textausschnitten (chunks) zu arbeiten, dem sog. Chunking. Es können die relativen Häufigkeiten der Wörter in einem Text genutzt werden, um die Häufigkeiten zu normalisieren. Oder alle Texte der zu vergleichende Korpora können auf die Länge des kürzesten Textes eines Korpus gebracht werden (truncating). Der letztgenannte Ansatz wird nachfolgend angewandt.

In [ ]:
def find_shortest_corpus(words_by_author):
    """Return length of shortest corpus."""
    word_count = []
    for author in words_by_author:
        word_count.append(len(words_by_author[author]))
        print('Anzahl der Tokens für {} = {}\n'.
              format(author, len(words_by_author[author])))
    len_shortest_corpus = min(word_count)
    print('Länge des kürzesten Korpus = {}\n'.format(len_shortest_corpus))        
    return len_shortest_corpus  

## Laden der Texte in ein Dictionary

In [ ]:
strings_by_author = dict()
strings_by_author['goethe'] = text_to_string('goethe.txt')
strings_by_author['humboldt'] = text_to_string('humboldt.txt')
strings_by_author['unknown'] = text_to_string('unknown.txt')

print(strings_by_author['goethe'][:300])

## Tokenisierung und Bestimmung des kürzesten Korpus

In [ ]:
words_by_author = make_word_dict(strings_by_author)
len_shortest_corpus = find_shortest_corpus(words_by_author)

## Anwenden der Teststrategien

### Häufigkeiten der Wortlängen

Durch das Auszählen und Visualisieren von Häufigkeiten der Wortlängen in einem ausreichend großen Textkorpus lässt sich mit Hilfe eines Liniendiagrammes eine charakteristische Kurve für Autor:innen eines Textkorpus veranschaulichen. Die Methode, Häufigkeiten von Wortlängen auszuzählen, wird schon länger angewandt, ist aber im Vergleich zu den aktuellen, viel elaborierten stilometrischen Methoden recht grob. Dennoch lassen sich mit diesem Ansatz durchaus erste Befunde gewinnen. 

Thomas C. Mendenhall, The Characteristic Curves of Composition, in: Science, vol. 9, no. 214 (Mar. 11, 1887), S. 237-249, https://doi.org/10.1126/science.ns-9.214s.237 / https://zenodo.org/record/1448355#.YlfoSNPP23A 

In [ ]:
# Linestyles für die Plots
LINES = ['-', ':', '--'] 

In [ ]:
def plot_word_length_test(words_by_author, len_shortest_corpus):
    """Plot word length freq by author, truncated to shortest corpus length."""
    
    fig, ax = plt.subplots(figsize=(10,6))
    for i, author in enumerate(words_by_author):
        word_lengths = [len(word) for word in words_by_author[author][:len_shortest_corpus] if len(word) > 2]
        by_author_length_freq_dist = nltk.FreqDist(word_lengths).most_common(15)

        ser_fdist = pd.Series(dict(by_author_length_freq_dist)) # Konvertieren in pandas-Series
        ser_fdist.sort_index().plot(ax=ax, 
                                    legend=True,
                                    linestyle=LINES[i],
                                    label=author,
                                    title='Vergleich: Häufigkeiten der Wortlänge',
                                    grid=True,
                                    xlabel='Wortlänge in Buchstaben',
                                    ylabel='Häufigkeit',
                                    xticks=range(3,18))

In [ ]:
plot_word_length_test(words_by_author, len_shortest_corpus)

### Häufigkeiten der Stoppwörter

Normalerweise werden Stoppwörter bei vielen Anwendungsszenarien des Natural Language Processing (NLP) aus einem Textkorpus entfernt, da sie nicht bedeutungstragend sind, daher wenig Information bieten und im Gegenteil ein stärkes Rauschen bei digitalen Textanalyseverfahren verursachen. Im Falle der stilometrischen Untersuchung von Texten hingegen erweisen sich die Stoppwörter als starkes Signal zur Bestimmung von Autorenschaft. Da Stoppwörter unbewusst und von Autor:innen in bestimmten Mustern genutzt werden, stellen sie eine gute Signatur dar, um Autor:innen zu identifzieren. Zwar lässt sich im nachfolgenden Diagramm eine Tendenz ablesen, jedoch nutzen ausgefeiltere stilometrischen Methoden Stoppwörter noch wesentlich effizienter.

In [ ]:
def plot_stopwords_test(words_by_author, len_shortest_corpus):
    """Plot stopwords freq by author, truncated to shortest corpus length."""

    fig, ax = plt.subplots(figsize=(10,5))

    AXES = [ax, ax.twiny(), ax.twiny()]
    COLORS = plt.get_cmap("tab10")
    SPINES_POSITION = [-.0, -.3, -.5] 

    stop_words = set(stopwords.words('german')) 
    print('Number of stopwords = {}\n'.format(len(stop_words)))

    for i, author in enumerate(words_by_author):

        stopwords_by_author = [word for word in words_by_author[author][:len_shortest_corpus] if word in stop_words]
        stopwords_by_author_freq_dist = nltk.FreqDist(stopwords_by_author).most_common(50)
        author_fdist = pd.Series(dict(stopwords_by_author_freq_dist))
        author_fdist.plot(ax=AXES[i], 
                        label=author,
                        legend=False,
                        linestyle=LINES[i],
                        title='Vergleich: Häufigkeiten der Stoppwörter',
                        color=COLORS(i),
                        ylabel='Häufigkeiten',
                        grid=True)
        AXES[i].spines['top'].set_position(('axes', SPINES_POSITION[i]))
        AXES[i].set_xticks(range(50))
        AXES[i].set_xticklabels(author_fdist.index, rotation=90, color=COLORS(i))
        AXES[i].spines['right'].set_visible(False)

    # add legend!

In [ ]:
plot_stopwords_test(words_by_author, len_shortest_corpus)

### Häufigkeiten der Wortarten (Part-of-Speech)

Ebenso lassen sich durch ein einfaches Auszählen der Wortarten ein weiterer Befund zum Stil eines:r Autors:in generieren, der im vorliegenden Fall allerdings nicht sehr deutlich ausfällt. Eine alphabetische List der Abkürzungen zu den verwendeten Tags ist [hier](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) zu finden.

In [ ]:
def plot_parts_of_speech_test(words_by_author, len_shortest_corpus):
    """Plot author use of parts-of-speech such as nouns, verbs, adverbs,etc."""

    fig, ax = plt.subplots(figsize=(10,5))

    AXES = [ ax, ax.twiny(), ax.twiny()]
    COLORS = plt.get_cmap("tab10")
    SPINES_POSITION = [ -.0, -.3, -.5]

    for i, author in enumerate(words_by_author):

        pos_by_author = [pos[1] for pos in nltk.pos_tag(words_by_author[author][:len_shortest_corpus])] 
        pos_by_author_freq_dist = nltk.FreqDist(pos_by_author).most_common(25)
        author_fdist = pd.Series(dict(pos_by_author_freq_dist))
        author_fdist.plot(ax=AXES[i], # oder sort_index() 
                        label=author,
                        legend=False,
                        linestyle=LINES[i],
                        title='Vergleich: Häufigkeiten nach Part-of-Speech-Tagging',
                        color=COLORS(i),
                        ylabel='Häufigkeiten',
                        grid=False)
        AXES[i].spines['top'].set_position(('axes', SPINES_POSITION[i]))
        AXES[i].set_xticks(range(len(author_fdist.index)))
        AXES[i].set_xticklabels(author_fdist.index, rotation=90, color=COLORS(i))
        AXES[i].spines['right'].set_visible(False)

In [ ]:
plot_parts_of_speech_test(words_by_author, len_shortest_corpus)

### Vergleich der am häufigsten verwendeten Wörter

In der nächsten Teststrategie wird die Distanz zwischen den Vokabularien unterschiedlicher Textkorpora mittels der Chi-Square Methode gemessen. Verglichen werden die Mengen der einmalig im jeweiligen Korpus auftretenden Wörter. Um die Distanz bzw. die Ähnlichkeit der Texte zu eruieren, wird gemessen, wie sehr sich die Anzahl der einzelnen Wörter der Vergleichskorpora unterscheiden. Je ähnlicher die Vokabularien sind, umso wahrscheinlicher ist es, dass ein:e Autor:in die beiden verglichenen Texte verfasst hat. Dabei wird davon ausgegangen, dass sich das Vokabular eines:r Autor:in und die Wortverwendung vergleichweise konstant ist. Das Vorgehen wird in einem Abschnitt der Programming Historian-Lesson zur Stilometrie [hier](https://programminghistorian.org/en/lessons/introduction-to-stylometry-with-python#second-stylometric-test-kilgariffs-chi-squared-method) näher beschrieben.

In [ ]:
def vocab_test(words_by_author):
    """Compare author vocabularies using the Chi Squared statistical test."""
    
    chisquared_by_author = dict()
    for author in words_by_author:
        if author != 'unknown': 
            # Combine corpus for author & unknown & find 1000 most-common words.
            combined_corpus = (words_by_author[author] +
                               words_by_author['unknown'])
            author_proportion = (len(words_by_author[author])/
                                 len(combined_corpus))
            combined_freq_dist = nltk.FreqDist(combined_corpus)
            most_common_words = list(combined_freq_dist.most_common(1000))
            chisquared = 0

            # Calculate observed vs. expected word counts.
            for word, combined_count in most_common_words:
                observed_count_author = words_by_author[author].count(word)
                expected_count_author = combined_count * author_proportion
                chisquared += ((observed_count_author -
                                expected_count_author)**2 /
                               expected_count_author)
                chisquared_by_author[author] = chisquared    
            print('Chi-squared für {} = {:.1f}'.format(author, chisquared))
            

    most_likely_author = min(chisquared_by_author, key=chisquared_by_author.get)
    print(f'Der wahrscheinlichste Autor nach Vergleich des Vokabulars: {most_likely_author}\n')

In [ ]:
vocab_test(words_by_author)

### Berechnung der Jaccard-Metrik

Auch mit dieser Metrik werden Distanz bzw. Ähnlichkeit von Textkorpora bestimmt. Der Wert wird errechnet, indem die Schnittmenge zweier Vokabularien durch die Verinigungsmenge beider Vokabularien geteilt wird. Die Metrik findet auch in  der englischen Bezeichnung Intersection over Union (IoU) verwendet. Je größer die Schnittmenge ist, um ähnlicher sind die Textkorpora und umso wahrscheinlicher stammen diese von einem:r Autor:in. 

In [ ]:
def jaccard_test(words_by_author, len_shortest_corpus):
    """Calculate Jaccard similarity of each known corpus to unknown corpus."""
    jaccard_by_author = dict()
    unique_words_unknown = set(words_by_author['unknown']
                               [:len_shortest_corpus])
    authors = (author for author in words_by_author if author != 'unknown')    
    for author in authors:
        unique_words_author = set(words_by_author[author][:len_shortest_corpus]) 
        shared_words = unique_words_author.intersection(unique_words_unknown)
        jaccard_sim = (float(len(shared_words))/ (len(unique_words_author) +
                                                  len(unique_words_unknown) -
                                                  len(shared_words)))
        jaccard_by_author[author] = jaccard_sim
        print('Jaccard Similarity für {} = {}'.format(author, jaccard_sim))
        
    most_likely_author = max(jaccard_by_author, key=jaccard_by_author.get)
    print(f'Der wahrscheinlichste Autor nach Jaccard Similarity: {most_likely_author}\n')

In [ ]:
jaccard_test(words_by_author, len_shortest_corpus) 